In [1]:
__author__ = 'Wzhou9'

In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.grid_search import RandomizedSearchCV

from sklearn.grid_search import GridSearchCV
from sklearn import cross_validation
from sklearn.metrics import mean_absolute_error
from time import time
from scipy.stats import randint as sp_randint

/Users/weijiezhou/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/weijiezhou/anaconda/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
train =pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
# Add target attribute "loss" in the test dataset and fulfill with Null
test["loss"]=np.nan

In [4]:
 joined = pd.concat([train, test])

In [5]:
def remove_st(col1,col2):
    set_1 = set(col1.unique())
    set_2 = set(col2.unique())
    rm = (set_2 - set_1).union(set_1 - set_2) 
    return rm

In [6]:
#df = pd.DataFrame([{'a': 15, 'b': 15, 'c': 5}, {'a': 20, 'b': 10, 'c': 7}, {'a': 25, 'b': 30, 'c': 9}])
#remove_st(df['a'],df['b'])


In [7]:
def filter_rm(col,rm):
    if col in rm:
        return -1
    return col
    

In [8]:
#def filter_rm(col,rm):
    #if col in rm:
        #return np.nan
    #return col

In [9]:
def handle_cat(train,test):
    #join two datasets
    joinDS = pd.concat([train, test])
    headercat = joinDS.select_dtypes(include = ['object']).columns
    #
    for i in headercat:
        #find remove set
        rm_list = remove_st(train[i], test[i])
        joinDS[i] = joinDS[i].apply(lambda x: filter_rm(x,rm_list), 1)
        joinDS[i] = pd.factorize(joined[i].values, sort=True)[0]    
    return joinDS   



In [10]:

joinDS = handle_cat(train,test)

In [11]:
train_a = joinDS[joinDS['loss'].notnull()]
test_a = joinDS[joinDS['loss'].isnull()]

In [12]:
#transform loss, input of xgboost
shift = 200
train_x = train_a.drop(['loss','id'],1)
train_label = np.log(train_a['loss'] + shift)
test_x = test_a.drop(['id','loss'],1)
test_id = test['id']

    

In [13]:
test_x.head(5)

,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
0,0,1,0,0,0,0,0,0,1,0,...,0.281143,0.466591,0.317681,0.61229,0.34365,0.38016,0.377724,0.369858,0.704052,0.392562
1,0,1,0,1,0,0,0,0,1,0,...,0.836443,0.482425,0.443760,0.71330,0.51890,0.60401,0.689039,0.675759,0.453468,0.208045
2,0,1,0,1,1,0,1,0,1,1,...,0.718531,0.212308,0.325779,0.29758,0.34365,0.30529,0.245410,0.241676,0.258586,0.297232
3,0,0,0,0,1,0,0,0,0,0,...,0.397069,0.369930,0.342355,0.40028,0.33237,0.31480,0.348867,0.341872,0.592264,0.555955
4,1,0,0,0,0,1,0,0,0,0,...,0.302678,0.398862,0.391833,0.23688,0.43731,0.50556,0.359572,0.352251,0.301535,0.825823


In [ ]:
model = RandomForestRegressor(max_depth = 6; n_jobs=-1,max_features = 0.2,n_estimators=70,random_state=16,oob_score = True,criterion = "mae",verbose = 2, min_samples_leaf = 50)

In [ ]:
model.fit(train_x.values, train_label.values)

In [ ]:
prediction_log = model.predict(test_x)
prediction = np.exp(prediction_log) - shift


In [ ]:
submission = pd.DataFrame()
submission['id'] = test_id
submission['loss'] = prediction
submission.to_csv('sub_v.csv', index=False)

In [14]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [20]:
param_dist = {"max_depth": [None],
             # "max_features": sp_randint(1, 11),
              #"min_samples_split": sp_randint(1, 11),
              #"min_samples_leaf": sp_randint(1, 11),
              #"bootstrap": [True, False],
               "criterion": ["mae"]
             }



In [21]:

random_search = RandomizedSearchCV(model, param_distributions=param_dist,
                                   n_iter=n_iter_search)



In [22]:
sklearn.__version__

'0.18.1'

In [23]:
start = time()
random_search.fit(train_x.values, train_label.values)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)

KeyboardInterrupt: 

In [ ]:
print ('MAE',mean_absolute_error(train_label.values, model.oob_prediction) )

In [ ]:


#Add the n_estimators value to the below list if you want to run the algo
n_list = numpy.array([])

for n_estimators in n_list:
    #Set the base model
    model = RandomForestRegressor(n_jobs=-1,n_estimators=1000,random_state=2016)
    
    algo = "RF"

    #Accuracy of the model using all features
    for name,i_cols_list in X_all:
        model.fit(X_train[:,i_cols_list],Y_train)
        result = mean_absolute_error(numpy.expm1(Y_val), numpy.expm1(model.predict(X_val[:,i_cols_list])))
        mae.append(result)
        print(name + " %s" % result)
        
    comb.append(algo + " %s" % n_estimators )

if (len(n_list)==0):
    mae.append(1213)
    comb.append("RF" + " %s" % 50 )    
    
##Set figure size
#plt.rc("figure", figsize=(25, 10))

##Plot the MAE of all combinations
#fig, ax = plt.subplots()
#plt.plot(mae)
##Set the tick names to names of combinations
#ax.set_xticks(range(len(comb)))
#ax.set_xticklabels(comb,rotation='vertical')
##Plot the accuracy for all combinations
#plt.show()    


In [ ]:
RANDOM_STATE = 2016
params = {
    'min_child_weight': 1,
    'eta': 0.01,
    'colsample_bytree': 0.5,
    'max_depth': 12,
    'subsample': 0.8,
    'alpha': 1,
    'gamma': 1,
    'silent': 1,
    'verbose_eval': True,
    'seed': RANDOM_STATE
    }

In [ ]:
def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    return 'mae', mean_absolute_error(np.exp(preds), np.exp(labels))

In [ ]:
xgtrain = xgb.DMatrix(train_x, label=train_label)
xgtest = xgb.DMatrix(test_x)
model = xgb.train(params, xgtrain, int(2012 / 0.9), feval=evalerror)
prediction = np.exp(model.predict(xgtest)) - shift



In [ ]:
submission = pd.DataFrame()
submission['id'] = test_id
submission['loss'] = prediction
submission.to_csv('sub_v.csv', index=False)

In [ ]:
x = pd.read_csv("sub_v.csv")

In [ ]:
x.head(5)